In [1]:
!pip install neural-semigroups

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [4]:
from neural_semigroups import Magma
from neural_semigroups.utils import hide_cells, partial_table_to_cube
import torch

cardinality = 8


def transform(x):
    if torch.randn((1, )).cpu().item() > 0.5:
        new_y = Magma(x[0]).random_isomorphism()
    else:
        new_y = Magma(x[0]).random_isomorphism().T
    new_x = partial_table_to_cube(
        hide_cells(
            new_y,
            cardinality * cardinality - cardinality
        )
    )
    return new_x,  partial_table_to_cube(new_y)

In [6]:
from neural_semigroups import Mace4Semigroups

data = Mace4Semigroups(
    root="2021.01.21.db",
    cardinality=cardinality,
    transform=transform
)

100%|██████████| 102711/102711 [00:07<00:00, 13191.17it/s]


In [7]:
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader

data_size = len(data)
print(data_size)
train_size = 1024
data_loaders = tuple(
    DataLoader(data_split, batch_size=32)
    for data_split
    in random_split(data, [train_size, train_size, data_size - 2 * train_size])
)

102711


In [8]:
from neural_semigroups.associator_loss import AssociatorLoss
from torch import Tensor

def loss(prediction: Tensor, target: Tensor) -> Tensor:
    return AssociatorLoss()(prediction)

In [9]:
from neural_semigroups import MagmaDAE

dae = MagmaDAE(
    cardinality=cardinality,
    hidden_dims=2 * [cardinality ** 3]
)

In [5]:
%reload_ext tensorboard

In [2]:
%tensorboard --logdir runs

Reusing TensorBoard on port 6006 (pid 453801), started 0:07:38 ago. (Use '!kill 453801' to kill it.)

In [10]:
!rm -rf runs

In [11]:
from neural_semigroups.training_helpers import learning_pipeline
from ignite.metrics.loss import Loss
from neural_semigroups.training_helpers import associative_ratio, guessed_ratio

params = {"learning_rate": 0.001, "epochs": 1000}
metrics = {
    "loss": Loss(loss),
    "associative_ratio": Loss(associative_ratio),
    "guessed_ratio": Loss(guessed_ratio)
}
learning_pipeline(params, dae, loss, metrics, data_loaders)

[1/1000]   0%|           [00:00<?]